## Read the script

In [ ]:
import os
import numpy as np
import mne
from ieeg.io import get_data, raw_from_layout, save_derivative
from ieeg.navigate import trial_ieeg, outliers_to_nan, channel_outlier_marker, crop_empty_data
from ieeg.calc.scaling import rescale
from ieeg.viz.ensemble import chan_grid
from ieeg.timefreq.utils import crop_pad, wavelet_scaleogram
from ieeg.viz.parula import parula_map
save_dir='C:\\Users\\bl314\\Box\\CoganLab\\IndividualMeetings\\Baishen\\ieeg_results\\lexical_delay'

In [ ]:
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab")
layout = get_data("LexicalDecRepDelay", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")
subject = 'D0063'

if subject=='D0053':
    subject_Tag = 'D53'
elif subject=='D0054':
    subject_Tag = 'D54'
elif subject=='D0055':
    subject_Tag = 'D55'
elif subject=='D0057':
    subject_Tag = 'D57'
elif subject=='D0059':
    subject_Tag = 'D59'
elif subject=='D0063':
    subject_Tag = 'D63'
elif subject=='D0065':
    subject_Tag = 'D65'
elif subject=='D0066':
    subject_Tag = 'D66'
elif subject=='D0068':
    subject_Tag = 'D68'
elif subject=='D0069':
    subject_Tag = 'D69'
elif subject=='D0070':
    subject_Tag = 'D70'
elif subject=='D0071':
    subject_Tag = 'D71'
elif subject=='D0077':
    subject_Tag = 'D77'
elif subject=='D0079':
    subject_Tag = 'D79'
elif subject=='D0081':
    subject_Tag = 'D81'
elif subject=='D0094':
    subject_Tag = 'D94'
elif subject=='D0096':
    subject_Tag = 'D96'
elif subject=='D0101':
    subject_Tag = 'D101'
elif subject=='D0102':
    subject_Tag = 'D102'
elif subject=='D0103':
    subject_Tag = 'D103'
elif subject=='D0107':
    subject_Tag = 'D107B'
else:
    print("Subject not found, please check.")

## Plot subj

In [ ]:
from ieeg.viz.mri import plot_subj
# plot the subject brain
fig1 = plot_subj(subject_Tag)
#Label every electrode
mne.viz.set_3d_view(fig1, azimuth=150, elevation=70, focalpoint="auto",
                    distance="auto")

## Load subj
### <span style="color:red">Please make sure that the **event.tsv** files in the **a** derivatives do not contain **"BAD boundary"** lines, or remove them if they have</span>

In [8]:
from rm_boundary_evnts import update_tsv 
tsv_loc = os.path.join(LAB_root, 'BIDS-1.0_LexicalDecRepDelay','BIDS','derivatives','a',f'sub-{subject}','ieeg')
update_tsv(subject,tsv_loc)

Processed and replaced the original file: C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0063\ieeg\sub-D0063_task-LexicalDecRepDelay_acq-01_run-01_desc-a_events.tsv
Processed and replaced the original file: C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0063\ieeg\sub-D0063_task-LexicalDecRepDelay_acq-01_run-02_desc-a_events.tsv
Processed and replaced the original file: C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0063\ieeg\sub-D0063_task-LexicalDecRepDelay_acq-01_run-03_desc-a_events.tsv
Processed and replaced the original file: C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0063\ieeg\sub-D0063_task-LexicalDecRepDelay_acq-01_run-04_desc-a_events.tsv


In [ ]:
raw = raw_from_layout(layout.derivatives['derivatives/a'], subject=subject, desc='a',extension='.edf',preload=True)

## Re-referenced to the average

In [ ]:
ch_type = raw.get_channel_types(only_data_chs=True)[0]
raw.set_eeg_reference(ref_channels="average", ch_type=ch_type)

## Wavelet spectrogram

In [ ]:
if not os.path.exists(os.path.join(save_dir, subject)):
    os.mkdir(os.path.join(save_dir, subject)) 
if not os.path.exists(os.path.join(save_dir, subject,'wavelet')):
    os.mkdir(os.path.join(save_dir, subject,'wavelet'))

In [ ]:
# Wavelet is good to detect and remove muscle artifact channels
# Also plot the subject brain
for task, task_Tag in zip(('Repeat','Yes_No'),('Rep','YN')):
    for epoch, t, tag in zip(
        ('Auditory_stim/'+task+'/CORRECT','Delay/'+task+'/CORRECT','Go/'+task+'/CORRECT','Resp/'+task+'/CORRECT'),
        ((-0.5, 1.5),(-0.5, 1.5),(-0.5, 1.5),(-0.5, 1)),
        ('Auditory-'+task_Tag,'Delay-'+task_Tag,'Go-'+task_Tag,'Resp-'+task_Tag)
    ):
        
        # Get the spectras
        t1 = t[0] - 0.5
        t2 = t[1] + 0.5
        times = (t1, t2)
        trials = trial_ieeg(raw, epoch, times, preload=True)
        outliers_to_nan(trials, outliers=10)
        
        ##############################
        ####### Wavelet ##############
        ##############################
        
        spectra_wavelet = wavelet_scaleogram(trials, n_jobs=-3, decim=int(
            raw.info['sfreq'] / 200)) # 1/10 of the timepionts, don't take too long
        crop_pad(spectra_wavelet, "0.5s") # cut the first and final 0.5s, change to zero
        
        # Get the baseline
        if epoch=='Auditory_stim/'+task+'/CORRECT':
            base_wavelet = spectra_wavelet.copy().crop(-0.5, 0)
            base_wavelet = base_wavelet.average(lambda x: np.nanmean(x, axis=0), copy=True)
        
        # Baseline correction
        spectra_wavelet = spectra_wavelet.average(lambda x: np.nanmean(x, axis=0), copy=True)   
        spectra_wavelet = rescale(spectra_wavelet, base_wavelet, copy=True, mode='ratio')
        spectra_wavelet._data = np.log10(spectra_wavelet._data) * 20
    
        # Save spectras
        filename = os.path.join(save_dir, subject,'wavelet',f'{tag}-tfr.h5')
        mne.time_frequency.write_tfrs(filename, spectra_wavelet, overwrite=True)
        
        # Make spectrograms
        chan_grids=chan_grid(spectra_wavelet, size = (20,10),vlim=(-2, 2), cmap=parula_map)
    
        # Save spectrograms
        fig_count=0
        for fig in chan_grids:
            figdir = os.path.join(save_dir, subject,'wavelet',f'{tag}_{fig_count+1}.jpg')
            chan_grids[fig_count].savefig(figdir,dpi=300)   
            fig_count+=1
            
        # Clean memory
        del spectra_wavelet,filename
  
    del base_wavelet